In [29]:
!pip install cbastats
!pip install pymysql
!pip install openpyxl

     |████████████████████████████████| 172 kB 1.6 MB/s eta 0:00:01
  Created wheel for openpyxl: filename=openpyxl-3.0.3-py2.py3-none-any.whl size=241262 sha256=98f3fe8ec3e1ca28adfdb01375d8df94f210412c30cf626f926a83eb319e7a81
  Stored in directory: /root/.cache/pip/wheels/5b/c0/e7/e16d3916984addf40bd021fa98d1624cb24a9adc48da4e62b5
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8915 sha256=7f9a23a9d8488a3ca613b84f4ae26feb87f1f032be0cb3bfab0b6a30886e4e07
  Stored in directory: /root/.cache/pip/wheels/e2/bd/55/048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31
Successfully built openpyxl et-xmlfile


In [1]:
from cbastats.Team import *
from cbastats.Player import *
from sqlalchemy import create_engine
import datetime

In [2]:
print('最近一次运行时间：',datetime.datetime.now())

最近一次运行时间： 2020-08-10 20:34:42.183720


# 载入数据

In [3]:
user_name = 'guest'
passcode = 'Guest123456'
endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
database = 'CBA_Data'
engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')

connection= engine.connect()

In [4]:
df = pd.read_sql("select * from CBA_Data.PlayerStatsPerGame", connection);

In [5]:
connection.close()

In [6]:
df.head().transpose()

,0,1,2,3,4
球员_link,http://cba.sports.sina.com.cn/cba/player/show/...,http://cba.sports.sina.com.cn/cba/player/show/...,http://cba.sports.sina.com.cn/cba/player/show/3/,http://cba.sports.sina.com.cn/cba/player/show/...,http://cba.sports.sina.com.cn/cba/player/show/...
球员,布鲁克斯,任骏飞,易建联,赵睿,胡明轩
号码,6,20,9,10,3
出场时间,36,30,35,31,13
首发,1,1,1,1,1
进攻篮板,0,1,1,2,0
防守篮板,3,1,11,5,1
助攻,9,1,2,4,0
犯规,3,3,2,3,1
抢断,2,3,1,2,0


In [9]:
whole_season = df
later_season = df.loc[df['轮次']>'第30轮'].copy()

# 所有球队高阶数据统计

In [8]:
# 整个赛季数据
# teams = Team('',whole_season)

# 仅复赛后数据
teams = Team('',later_season)


teams_df = pd.concat([teams.mov, teams.tm_pace, teams.tm_ortg, teams.tm_drtg, teams.tm_nrtg], axis=1)
teams_df.columns=['场均净胜分MOV', 'Pace', 'OffensiveRating', 'DefensiveRating', 'NetRating']
teams_df = teams_df.sort_values(by='NetRating',ascending=False)
teams_df.round(1)

,场均净胜分MOV,Pace,OffensiveRating,DefensiveRating,NetRating
球队,,,,,
广东,20.1,98.8,126.6,105.8,20.8
辽宁,17.4,95.4,126.1,107.5,18.6
新疆,14.2,94.3,118.8,103.5,15.3
北京,8.7,88.7,110.9,100.9,10.0
广厦,6.7,94.0,112.4,105.1,7.3
浙江,5.8,91.4,116.5,110.0,6.5
福建,0.4,99.1,117.5,117.1,0.4
青岛,-1.6,93.9,111.2,113.0,-1.7
北控,-1.6,92.8,107.1,108.9,-1.8


# Model Coeficients 

Reference:
* https://medium.com/hackernoon/how-to-predict-the-nba-with-a-machine-learning-system-written-in-python-part-ii-f276b19520b9
* https://www.basketball-reference.com/about/factors.html
* https://www.basketball-reference.com/about/glossary.html

In [ ]:
coefs = dict({})

# home court advantage: 0.10218887
# 因为是隔离比赛，所以没有主场优势
coefs['home_court_advantage'] = 0

In [ ]:
# effective_field_goal_percentage_difference
coefs['effective_field_goal_percentage_difference']= 0.16118265
tm_3pg = teams.all_games_stats.groupby('球队')['3分中'].sum()
effective_field_goal_percentage = 100*(teams.tm_fg + 0.5 * tm_3pg) / teams.tm_fga

In [ ]:
# coef_turnover_percentage_difference
coefs['turnover_percentage_difference']=-0.05958713
tm_tov = teams.all_games_stats.groupby('球队')['失误'].sum()
# below definition is from BBR and 4 factors by Dean Oliver
turnover_percentage = 100*tm_tov / (teams.tm_fga + 0.44 * teams.tm_fta + tm_tov)

In [ ]:
# coef_offensive_rebound_percentage_difference
coefs['offensive_rebound_percentage_difference']=0.07061777
# below definition is from BBR
# offensive_rebound_percentage = 100 * (teams.tm_orb * (teams.tm_mp / 5)) / (teams.tm_mp*2 * (teams.tm_orb + teams.op_tm_orb))

# below definition is from 4 factors by Dean Oliver
offensive_rebound_percentage = 100*(teams.tm_orb/(teams.tm_orb+teams.op_tm_drb))

In [ ]:
coefs['free_throws_to_field_goals_attempts_difference']=0.03267933
tm_ftg = teams.all_games_stats.groupby('球队')['罚球中'].sum()

# author didn't say but in 4 factors it's a percentage, 
# we'll decide later when result is out
free_throws_to_field_goals_attempts = 100*tm_ftg/teams.tm_fga

In [ ]:
# no one is traveling
# coefs['distance_traveled_in_last_7_days_difference']=-0.01459163
coefs['distance_traveled_in_last_7_days_difference']=0

In [ ]:
# looks like it's win lose last seven matches per https://www.transfermarkt.us/premier-league/formtabelle/wettbewerb/GB1
# coefs['form_in_last_seven_matches_difference']=0.0828436
coefs['form_in_last_seven_matches_difference']=0

In [ ]:
coefs['offensive_rating_difference']=0.17885523
offensive_rating = teams.tm_ortg

In [ ]:
coefs['defensive_rating_difference']=-0.33924331
defensive_rating = teams.tm_drtg

In [ ]:
# no court advantage at this moment

# coefs['effective_field_goal_percentage_difference_court']=0.10808104
# coefs['turnover_percentage_difference_court']=-0.09548481
# coefs['offensive_rebound_percentage_difference_court']=0.07055131
# coefs['free_throws_to_field_goals_attempts_difference_court']=0.0748545
# coefs['form_in_last_seven_matches_difference_court']=-0.00486437
# coefs['offensive_rating_difference_court']=0.14822224
# coefs['defensive_rating_difference_court']=-0.21756487

In [145]:
import math

# define this function to calculate probability
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [178]:
list(translation.values())

['真实命中率', '失误率', '进攻篮板率', '罚球率', '进攻效率', '防守效率']

In [179]:
schedules = [('青岛','山东'),
('广厦','福建'),
('北控','吉林'),
('浙江','山西')]

translation = dict({'effective_field_goal_percentage_difference':'真实命中率',
                   'turnover_percentage_difference':'失误率',
                   'offensive_rebound_percentage_difference':'进攻篮板率',
                   'free_throws_to_field_goals_attempts_difference':'罚球率',
                   'offensive_rating_difference':'进攻效率',
                   'defensive_rating_difference':'防守效率'})

dfs = []

for schedule in schedules:
    home_team = schedule[0]
    away_team = schedule[1]
    home_factors = []
    away_factors = []
    differences = []
    

    Xs = coefs.copy()
    Xs['home_court_advantage'] = 0
    Xs['effective_field_goal_percentage_difference'] = effective_field_goal_percentage[home_team]-effective_field_goal_percentage[away_team]
    Xs['turnover_percentage_difference']= turnover_percentage[home_team] - turnover_percentage[away_team]
    Xs['offensive_rebound_percentage_difference']= offensive_rebound_percentage[home_team]-offensive_rebound_percentage[away_team]
    Xs['free_throws_to_field_goals_attempts_difference'] = free_throws_to_field_goals_attempts[home_team]-free_throws_to_field_goals_attempts[away_team]
    Xs['distance_traveled_in_last_7_days_difference']= 0
    Xs['form_in_last_seven_matches_difference']= 0
    Xs['offensive_rating_difference'] = offensive_rating[home_team] - offensive_rating[away_team]
    Xs['defensive_rating_difference'] = defensive_rating[home_team] - defensive_rating[away_team]
    
    Z = 0
    for key in coefs:
        Z+= coefs[key]*Xs[key]
    
    
    print(home_team+' V.S '+away_team)
    print(f'{home_team}获胜概率：'+str(sigmoid(Z)))
    print()
    for key in coefs:
        if key not in ['home_court_advantage','distance_traveled_in_last_7_days_difference','form_in_last_seven_matches_difference']:
            print(f"{translation[key]}: {eval(key[:-11]+'[home_team]').round(2)}-{eval(key[:-11]+'[away_team]').round(2)} = {Xs[key].round(2)}")
            home_factors.append(eval(key[:-11]+'[home_team]').round(2))
            away_factors.append(eval(key[:-11]+'[away_team]').round(2))
            differences.append(Xs[key].round(2))
    
    dfs.append(pd.DataFrame({home_team:home_factors,away_team:away_factors,'差值':differences},
                            index=list(translation.values())))
    
    print('-----------------------')

青岛 V.S 山东
青岛获胜概率：0.29892010501084254

真实命中率: 47.62-51.43 = -3.81
失误率: 14.93-17.97 = -3.04
进攻篮板率: 27.64-29.22 = -1.59
罚球率: 23.85-21.45 = 2.4
进攻效率: 103.06-102.6 = 0.46
防守效率: 109.81-108.43 = 1.38
-----------------------
广厦 V.S 福建
广厦获胜概率：0.9394443523572833

真实命中率: 51.51-54.4 = -2.89
失误率: 14.94-13.24 = 1.71
进攻篮板率: 33.18-25.36 = 7.82
罚球率: 26.03-24.63 = 1.4
进攻效率: 111.87-114.62 = -2.75
防守效率: 102.81-112.25 = -9.44
-----------------------
北控 V.S 吉林
北控获胜概率：0.7907049616637225

真实命中率: 48.71-52.53 = -3.82
失误率: 15.46-12.81 = 2.65
进攻篮板率: 32.68-23.82 = 8.87
罚球率: 27.5-20.49 = 7.01
进攻效率: 107.69-109.49 = -1.8
防守效率: 110.39-115.01 = -4.63
-----------------------
浙江 V.S 山西
浙江获胜概率：0.9945878509482465

真实命中率: 52.75-50.51 = 2.24
失误率: 14.23-15.23 = -1.0
进攻篮板率: 30.89-21.79 = 9.1
罚球率: 24.61-26.01 = -1.4
进攻效率: 114.38-103.06 = 11.32
防守效率: 104.22-110.62 = -6.4
-----------------------


In [183]:
pd.concat(dfs,axis=1)

,青岛,山东,差值,广厦,福建,差值,北控,吉林,差值,浙江,山西,差值
真实命中率,47.62,51.43,-3.81,51.51,54.40,-2.89,48.71,52.53,-3.82,52.75,50.51,2.24
失误率,14.93,17.97,-3.04,14.94,13.24,1.71,15.46,12.81,2.65,14.23,15.23,-1.00
进攻篮板率,27.64,29.22,-1.59,33.18,25.36,7.82,32.68,23.82,8.87,30.89,21.79,9.10
罚球率,23.85,21.45,2.40,26.03,24.63,1.40,27.50,20.49,7.01,24.61,26.01,-1.40
进攻效率,103.06,102.60,0.46,111.87,114.62,-2.75,107.69,109.49,-1.80,114.38,103.06,11.32
防守效率,109.81,108.43,1.38,102.81,112.25,-9.44,110.39,115.01,-4.63,104.22,110.62,-6.40


In [184]:
for x in dfs:
    print(x)

           青岛      山东    差值
真实命中率   47.62   51.43 -3.81
失误率     14.93   17.97 -3.04
进攻篮板率   27.64   29.22 -1.59
罚球率     23.85   21.45  2.40
进攻效率   103.06  102.60  0.46
防守效率   109.81  108.43  1.38
           广厦      福建    差值
真实命中率   51.51   54.40 -2.89
失误率     14.94   13.24  1.71
进攻篮板率   33.18   25.36  7.82
罚球率     26.03   24.63  1.40
进攻效率   111.87  114.62 -2.75
防守效率   102.81  112.25 -9.44
           北控      吉林    差值
真实命中率   48.71   52.53 -3.82
失误率     15.46   12.81  2.65
进攻篮板率   32.68   23.82  8.87
罚球率     27.50   20.49  7.01
进攻效率   107.69  109.49 -1.80
防守效率   110.39  115.01 -4.63
           浙江      山西     差值
真实命中率   52.75   50.51   2.24
失误率     14.23   15.23  -1.00
进攻篮板率   30.89   21.79   9.10
罚球率     24.61   26.01  -1.40
进攻效率   114.38  103.06  11.32
防守效率   104.22  110.62  -6.40


# 用整个赛季数据预测：

    青岛 V.S 山东
    0.41543370748160735
    广厦 V.S 福建
    0.9125730862476105
    北控 V.S 吉林
    0.6327203272535492
    浙江 V.S 山西
    0.8515366750830545

# 用复赛后数据预测：

    青岛 V.S 山东
    0.29892010501084254
    广厦 V.S 福建
    0.9394443523572833
    北控 V.S 吉林
    0.7907049616637225
    浙江 V.S 山西
    0.9945878509482465

# 总结

总体思路参考了Dean Oliver 的四要素，Shooting, Turnovers, Rebounding, Free Throws

使用数据：

* 有效命中率 effective_field_goal_percentage_difference：The formula for both offense and defense is (FG + 0.5 * 3P) / FGA.

* 失误率turnover_percentage_difference:The formula for both offense and defense is TOV / (FGA + 0.44 * FTA + TOV)

* 进攻篮板率offensive_rebound_percentage_difference（此处BBR和Dean Oliver有不同定义，考虑到文章作者提到了Dean Oliver的4 facor，此处使用的是Dean Oliver的定义）: The formula for offense is ORB / (ORB + Opp DRB), while the formula for defense is DRB / (Opp ORB + DRB).

* 罚球率？？free_throws_to_field_goals_attempts_difference: The free throw factor is a measure of both how often a team gets to the line and how often they make them

另外还用到了：
* 进攻效率offensive_rating_difference

* 防守效率defensive_rating_difference